# 🏠 Iowa Housing — Regression Pipeline

**Goal:** Predict the sale price of houses in Ames, Iowa using 80+ features.  
**Dataset:** [Kaggle House Prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques) — 1460 training samples.  
**Approach:** Full ML pipeline → EDA → Feature Engineering → Model Comparison → Tuning → Interpretation → Submission.

---
## 0. Configuration & Imports

Centralising every constant at the top makes the notebook reproducible and easy to tweak.  
Creating output directories up front prevents file-not-found errors when saving plots later.  
Importing everything here keeps the rest of the notebook clean.

In [ ]:
# ── Constants ────────────────────────────────────────────────────────
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import time

SEED       = 42
TEST_SIZE  = 0.2
CV_FOLDS   = 5
TARGET_COL = "SalePrice"

DATA_DIR    = Path("data")
OUTPUT_DIR  = Path("outputs")
FIGURES_DIR = OUTPUT_DIR / "figures"
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# ── Libraries ────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import (
    train_test_split, cross_val_score, KFold, GridSearchCV,
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# ── Reproducibility & style ──────────────────────────────────────────
np.random.seed(SEED)
plt.style.use("seaborn-v0_8")
plt.rcParams.update({"figure.figsize": (10, 6), "axes.titlesize": 14, "axes.labelsize": 12})

print("Setup complete ✅")

---
## 1. Load Data & First Inspection

Loading the raw data and immediately checking shape, types, and summary statistics gives us a mental model of the dataset.  
Separating numeric vs categorical columns up front is essential because they require completely different processing pipelines.

In [ ]:
# ── Load datasets ────────────────────────────────────────────────────
df_train = pd.read_csv(DATA_DIR / "train.csv")
df_test  = pd.read_csv(DATA_DIR / "test.csv")

# Preserve test IDs for submission
test_ids = df_test["Id"].copy()

print(f"Train: {df_train.shape}  |  Test: {df_test.shape}")
df_train.head(5)

In [ ]:
# ── Data types overview ──────────────────────────────────────────────
print(df_train.dtypes.value_counts())
print("\n")
df_train.describe()

In [ ]:
# ── Separate feature types ───────────────────────────────────────────
# MSSubClass is numeric in dtype but categorical in meaning
df_train["MSSubClass"] = df_train["MSSubClass"].astype(str)
df_test["MSSubClass"]  = df_test["MSSubClass"].astype(str)

NUMERIC_COLS     = df_train.select_dtypes(include=np.number).columns.drop(["Id", TARGET_COL]).tolist()
CATEGORICAL_COLS = df_train.select_dtypes(include="object").columns.tolist()

print(f"Numeric features:     {len(NUMERIC_COLS)}")
print(f"Categorical features: {len(CATEGORICAL_COLS)}")
print(f"Target range: ${df_train[TARGET_COL].min():,.0f} – ${df_train[TARGET_COL].max():,.0f}")

### 📌 Summary
- **36 numeric** and **44 categorical** features (after casting `MSSubClass` to string).
- Target `SalePrice` ranges from ~\$35 k to ~\$755 k.

---
## 2. Target Variable Analysis

Many regression metrics (including RMSE on Kaggle) are evaluated on the **log** of the target.  
A log-transform makes the right-skewed SalePrice distribution closer to Normal, which satisfies the assumptions of linear models and stabilises variance.

In [ ]:
# ── Plot 1: Raw SalePrice distribution ──────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1a. Raw histogram + KDE
sns.histplot(df_train[TARGET_COL], kde=True, ax=axes[0], color="steelblue")
axes[0].set_title("SalePrice Distribution (Raw)")
axes[0].set_xlabel("SalePrice ($)")
axes[0].set_ylabel("Count")
axes[0].axvline(df_train[TARGET_COL].mean(), color="red", ls="--", label=f"Mean: ${df_train[TARGET_COL].mean():,.0f}")
axes[0].legend()

# 1b. Log-transformed histogram + KDE
log_price = np.log1p(df_train[TARGET_COL])
sns.histplot(log_price, kde=True, ax=axes[1], color="#2ecc71")
axes[1].set_title("log(1 + SalePrice) Distribution")
axes[1].set_xlabel("log(1 + SalePrice)")
axes[1].set_ylabel("Count")

# 1c. QQ-plot
stats.probplot(log_price, dist="norm", plot=axes[2])
axes[2].set_title("QQ-Plot: log(SalePrice) vs Normal")

plt.tight_layout()
fig.savefig(FIGURES_DIR / "target_distribution.png", dpi=150)
plt.show()

print(f"Skewness (raw):  {df_train[TARGET_COL].skew():.3f}")
print(f"Skewness (log):  {log_price.skew():.3f}")
print(f"Kurtosis (log):  {log_price.kurtosis():.3f}")

In [ ]:
# ── Apply log1p transform to the target ─────────────────────────────
y_train_log = np.log1p(df_train[TARGET_COL])

- Raw `SalePrice` has **positive skew ≈ 1.88** — the right tail of expensive houses pulls the mean above the median.
- After `log1p`, skewness drops to **≈ 0.12** and the QQ-plot is nearly linear → safe for linear models.
- We will predict `log(1+SalePrice)` during training and back-transform with `expm1` at submission.

---
## 3. Missing Data Analysis & Strategy

The Iowa Housing dataset uses `NaN` for both *truly missing* data and *absence of a feature* (e.g., "no pool").  
Domain-aware imputation is critical: filling `PoolQC = NaN` with "None" (no pool) is semantically correct, whereas using the mode would be wrong.  
We must handle both sets identically to avoid train–test mismatch.

In [ ]:
# ── Missing-value summary for both sets ─────────────────────────────
def missing_summary(df: pd.DataFrame, name: str) -> pd.DataFrame:
    """Return a DataFrame of columns with missing values and their percentage."""
    total = df.isnull().sum()
    pct   = (total / len(df) * 100).round(1)
    out   = pd.DataFrame({"count": total, "pct": pct}).query("count > 0").sort_values("pct", ascending=False)
    out.columns = [f"{name}_count", f"{name}_pct"]
    return out

miss_train = missing_summary(df_train, "train")
miss_test  = missing_summary(df_test,  "test")
miss_all   = miss_train.join(miss_test, how="outer").fillna(0)
print(f"Columns with missing values — train: {len(miss_train)}, test: {len(miss_test)}")
miss_all.head(20)

In [ ]:
# ── Plot: Top 30 columns by missing % (train) ───────────────────────
fig, ax = plt.subplots(figsize=(10, 8))
top30 = miss_train.head(30)
ax.barh(top30.index, top30["train_pct"], color="salmon")
ax.set_xlabel("Missing (%)")
ax.set_ylabel("Feature")
ax.set_title("Top 30 Features by Missing Percentage (Train)")
ax.invert_yaxis()
plt.tight_layout()
fig.savefig(FIGURES_DIR / "missing_values.png", dpi=150)
plt.show()

In [ ]:
# ── Domain-aware imputation ─────────────────────────────────────────
# These columns use NaN to mean "feature absent", not "unknown"
NONE_FILL_COLS = [
    "PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu",
    "GarageType", "GarageFinish", "GarageQual", "GarageCond",
    "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2",
    "MasVnrType",
]

# Numeric cols where NaN means zero
ZERO_FILL_COLS = [
    "GarageYrBlt", "GarageArea", "GarageCars",
    "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF",
    "BsmtFullBath", "BsmtHalfBath", "MasVnrArea",
]

# Columns to drop (>40 % missing)
HIGH_MISSING = [c for c in miss_train.index if miss_train.loc[c, "train_pct"] > 40]
print(f"Dropping (>40 % missing): {HIGH_MISSING}")


def impute_missing(df: pd.DataFrame) -> pd.DataFrame:
    """Apply domain-aware imputation to all missing columns."""
    df = df.copy()

    # Drop high-missing columns
    df.drop(columns=[c for c in HIGH_MISSING if c in df.columns], inplace=True, errors="ignore")

    # Fill "absence" categoricals with "None"
    for col in NONE_FILL_COLS:
        if col in df.columns:
            df[col] = df[col].fillna("None")

    # Fill "absence" numerics with 0
    for col in ZERO_FILL_COLS:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    # Fix GarageYrBlt typo (2207 → 2007)
    if "GarageYrBlt" in df.columns:
        df["GarageYrBlt"] = df["GarageYrBlt"].clip(upper=2010)

    # Remaining categoricals → mode
    for col in df.select_dtypes(include="object").columns:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mode()[0])

    # Remaining numerics → median
    for col in df.select_dtypes(include=np.number).columns:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].median())

    return df


df_train = impute_missing(df_train)
df_test  = impute_missing(df_test)

# ── Assert no nulls remain ───────────────────────────────────────────
assert df_train.isnull().sum().sum() == 0, "Train still has nulls!"
assert df_test.isnull().sum().sum()  == 0, "Test still has nulls!"
print(f"✅ Zero nulls remaining — train: {df_train.shape}, test: {df_test.shape}")

### 📌 Before / After

| Bucket | Columns | Strategy |
|--------|---------|----------|
| **HIGH (>40 %)** | PoolQC, MiscFeature, Alley, Fence | **Dropped** |
| **Semantic NaN** | FireplaceQu, Garage*, Bsmt*, MasVnrType | Filled with **"None"** / **0** |
| **Low missing** | LotFrontage, Electrical, etc. | Median / Mode |

---
## 4. Exploratory Data Analysis — 10 Charts

EDA reveals which features carry the most signal for predicting SalePrice.  
We identify outliers, confirm expected domain relationships (quality → price), and spot high-cardinality categoricals.  
Every chart is saved to `outputs/figures/` for the report.

In [ ]:
# ── Chart 1: Top 10 numeric correlations with SalePrice ─────────────
numeric_for_corr = [c for c in NUMERIC_COLS if c in df_train.columns] + [TARGET_COL]
corr = df_train[numeric_for_corr].corr()
top10 = corr[TARGET_COL].drop(TARGET_COL).abs().nlargest(10).index.tolist() + [TARGET_COL]

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(df_train[top10].corr(), annot=True, fmt=".2f", cmap="RdBu_r",
            center=0, square=True, linewidths=0.5, ax=ax)
ax.set_title("Top 10 Features Correlated with SalePrice")
plt.tight_layout()
fig.savefig(FIGURES_DIR / "correlation_top10.png", dpi=150)
plt.show()

- **OverallQual** (0.79) and **GrLivArea** (0.71) have the strongest positive correlations.
- **GarageCars / GarageArea** are highly correlated with each other (~0.88) — we might drop one to avoid multicollinearity.

In [ ]:
# ── Chart 2: SalePrice vs GrLivArea scatter ─────────────────────────
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(df_train["GrLivArea"], df_train[TARGET_COL],
           alpha=0.4, c="steelblue", edgecolors="k", linewidths=0.3, s=30)

# Mark outliers: GrLivArea > 4000
outliers = df_train[df_train["GrLivArea"] > 4000]
ax.scatter(outliers["GrLivArea"], outliers[TARGET_COL],
           c="red", s=80, edgecolors="k", label=f"Outliers ({len(outliers)})")
ax.set_title("SalePrice vs GrLivArea")
ax.set_xlabel("Above-Ground Living Area (sq ft)")
ax.set_ylabel("SalePrice ($)")
ax.legend()
plt.tight_layout()
fig.savefig(FIGURES_DIR / "scatter_grliv.png", dpi=150)
plt.show()

- Two houses with **GrLivArea > 4500** but low SalePrice are clear outliers (likely partial sales or data errors).
- We will **remove** these before training.

In [ ]:
# ── Remove GrLivArea outliers ────────────────────────────────────────
before = len(df_train)
df_train = df_train[df_train["GrLivArea"] <= 4500].reset_index(drop=True)
y_train_log = np.log1p(df_train[TARGET_COL])
print(f"Removed {before - len(df_train)} outliers → {len(df_train)} rows remain")

In [ ]:
# ── Chart 3: SalePrice vs OverallQual (violin) ──────────────────────
fig, ax = plt.subplots(figsize=(12, 5))
sns.violinplot(x="OverallQual", y=TARGET_COL, data=df_train, ax=ax, palette="YlOrRd", inner="quartile")
ax.set_title("SalePrice Distribution by Overall Quality")
ax.set_xlabel("Overall Quality (1–10)")
ax.set_ylabel("SalePrice ($)")
plt.tight_layout()
fig.savefig(FIGURES_DIR / "violin_quality.png", dpi=150)
plt.show()

- Price increases roughly **exponentially** with quality: quality 10 houses sell for 3–5× more than quality 5.
- Strong separation → OverallQual will be a top predictor.

In [ ]:
# ── Chart 4: SalePrice by Neighborhood (boxplot, sorted) ────────────
nbh_order = df_train.groupby("Neighborhood")[TARGET_COL].median().sort_values().index

fig, ax = plt.subplots(figsize=(16, 6))
sns.boxplot(x="Neighborhood", y=TARGET_COL, data=df_train,
            order=nbh_order, ax=ax, palette="viridis")
ax.set_title("SalePrice by Neighborhood (sorted by median)")
ax.set_xlabel("Neighborhood")
ax.set_ylabel("SalePrice ($)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
fig.savefig(FIGURES_DIR / "boxplot_neighborhood.png", dpi=150)
plt.show()

- **NoRidge, NridgHt, StoneBr** are the most expensive neighbourhoods (median > $250 k).
- **MeadowV, IDOTRR, BrDale** are the cheapest (median < $100 k).

In [ ]:
# ── Chart 5: SalePrice by decade built ──────────────────────────────
df_train["Decade"] = (df_train["YearBuilt"] // 10) * 10
decade_price = df_train.groupby("Decade")[TARGET_COL].mean()

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(decade_price.index, decade_price.values, "o-", color="steelblue", lw=2)
ax.set_title("Mean SalePrice by Decade Built")
ax.set_xlabel("Decade")
ax.set_ylabel("Mean SalePrice ($)")
ax.grid(True, alpha=0.3)
plt.tight_layout()
fig.savefig(FIGURES_DIR / "line_decade.png", dpi=150)
plt.show()
df_train.drop(columns=["Decade"], inplace=True)

- Houses built after 2000 command significantly higher prices (newer construction, modern amenities).
- Pre-1920 homes retain some value — likely historic/character appeal.

In [ ]:
# ── Charts 6 & 7: SalePrice vs TotalBsmtSF and GarageArea ─────────
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, col, color in zip(axes, ["TotalBsmtSF", "GarageArea"], ["#3498db", "#e67e22"]):
    ax.scatter(df_train[col], df_train[TARGET_COL], alpha=0.4, c=color, edgecolors="k", linewidths=0.3, s=25)
    ax.set_title(f"SalePrice vs {col}")
    ax.set_xlabel(col)
    ax.set_ylabel("SalePrice ($)")

plt.tight_layout()
fig.savefig(FIGURES_DIR / "scatter_bsmt_garage.png", dpi=150)
plt.show()

- Both features show a **positive linear trend** with SalePrice.
- `TotalBsmtSF = 0` cluster represents houses with no basement.

In [ ]:
# ── Chart 8: Distribution of OverallQual ────────────────────────────
fig, ax = plt.subplots(figsize=(8, 4))
df_train["OverallQual"].value_counts().sort_index().plot.bar(ax=ax, color="steelblue", edgecolor="k")
ax.set_title("Distribution of Overall Quality")
ax.set_xlabel("OverallQual")
ax.set_ylabel("Count")
plt.tight_layout()
fig.savefig(FIGURES_DIR / "hist_overallqual.png", dpi=150)
plt.show()

- Most houses cluster around quality **5–7** (average to above-average).
- Only a handful are rated 1–2 or 10 — extreme tails.

In [ ]:
# ── Chart 9: SalePrice by MSZoning (hue = low/high price) ──────────
median_price = df_train[TARGET_COL].median()
df_train["PriceBin"] = np.where(df_train[TARGET_COL] >= median_price, "High", "Low")

fig, ax = plt.subplots(figsize=(8, 5))
sns.countplot(x="MSZoning", hue="PriceBin", data=df_train, ax=ax,
              palette={"Low": "#e74c3c", "High": "#2ecc71"}, hue_order=["Low", "High"])
ax.set_title("MSZoning Count (Low vs High Price)")
ax.set_xlabel("MSZoning")
ax.set_ylabel("Count")
plt.tight_layout()
fig.savefig(FIGURES_DIR / "bar_mszoning.png", dpi=150)
plt.show()

df_train.drop(columns=["PriceBin"], inplace=True)

- **RL (Residential Low Density)** dominates the dataset and has balanced high/low sales.
- **RM (Residential Medium)** skews towards lower prices.

In [ ]:
# ── Chart 10: Pairplot (top 5 features + SalePrice, sampled) ───────
top5_feats = ["OverallQual", "GrLivArea", "TotalBsmtSF", "GarageArea", "YearBuilt", TARGET_COL]
sample = df_train[top5_feats].sample(300, random_state=SEED)

g = sns.pairplot(sample, corner=True, plot_kws={"alpha": 0.4, "s": 15})
g.figure.suptitle("Pairplot — Top 5 Features + SalePrice", y=1.01)
g.figure.savefig(FIGURES_DIR / "pairplot_top5.png", dpi=100, bbox_inches="tight")
plt.show()

- **GrLivArea** and **TotalBsmtSF** show clear linear relationships with SalePrice.
- **YearBuilt** vs SalePrice shows a non-linear (accelerating) trend for newer homes.

---
## 5. Feature Engineering

Deriving composite features like `TotalSF` captures information spread across multiple columns.  
Ordinal encoding preserves the natural order in quality scales; one-hot handles unordered categories.  
Correcting skewness in numeric features prevents a few extreme values from dominating tree splits and linear weights.

In [ ]:
# ── Ordinal quality mapping ──────────────────────────────────────────
QUAL_MAP = {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}
QUAL_COLS = [
    "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC",
    "KitchenQual", "FireplaceQu", "GarageQual", "GarageCond",
]


def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """Create new features, encode categoricals, and fix skewness."""
    df = df.copy()

    # ── Derived numeric features ─────────────────────────────────────
    df["TotalSF"]        = df["TotalBsmtSF"] + df["1stFlrSF"] + df["2ndFlrSF"]
    df["TotalBathrooms"] = (df["FullBath"] + 0.5 * df["HalfBath"]
                            + df["BsmtFullBath"] + 0.5 * df["BsmtHalfBath"])
    df["TotalPorchSF"]   = (df["OpenPorchSF"] + df["EnclosedPorch"]
                            + df["3SsnPorch"] + df["ScreenPorch"])
    df["HouseAge"]       = df["YrSold"] - df["YearBuilt"]
    df["RemodelAge"]     = df["YrSold"] - df["YearRemodAdd"]
    df["IsRemodeled"]    = (df["YearRemodAdd"] != df["YearBuilt"]).astype(int)
    df["GarageAge"]      = (df["YrSold"] - df["GarageYrBlt"]).clip(lower=0).fillna(0)
    df["HasPool"]        = (df.get("PoolArea", pd.Series(0, index=df.index)) > 0).astype(int)
    df["Has2ndFloor"]    = (df["2ndFlrSF"] > 0).astype(int)
    df["HasGarage"]      = (df["GarageArea"] > 0).astype(int)
    df["HasBsmt"]        = (df["TotalBsmtSF"] > 0).astype(int)
    df["HasFireplace"]   = (df["Fireplaces"] > 0).astype(int)

    # ── Ordinal encode quality columns ───────────────────────────────
    for col in QUAL_COLS:
        if col in df.columns:
            df[col] = df[col].map(QUAL_MAP).fillna(0).astype(int)

    # ── Drop Id and raw columns we've replaced ──────────────────────
    df.drop(columns=["Id"], inplace=True, errors="ignore")

    # ── One-hot encode remaining categoricals ────────────────────────
    df = pd.get_dummies(df, drop_first=True, dtype=int)

    return df


print(f"Before engineering — train: {df_train.shape}, test: {df_test.shape}")

# Keep target separate
train_eng = engineer_features(df_train.drop(columns=[TARGET_COL]))
test_eng  = engineer_features(df_test)

# Align columns (add missing one-hot cols as 0, drop extras)
train_eng, test_eng = train_eng.align(test_eng, join="left", axis=1, fill_value=0)

print(f"After engineering  — train: {train_eng.shape}, test: {test_eng.shape}")

In [ ]:
# ── Skewness correction (log1p for features with skew > 0.75) ──────
numeric_feats = train_eng.select_dtypes(include=np.number).columns
skew_before = train_eng[numeric_feats].apply(lambda x: x.skew()).sort_values(ascending=False)
skewed = skew_before[skew_before.abs() > 0.75]
print(f"Features with |skew| > 0.75: {len(skewed)}")

for col in skewed.index:
    if (train_eng[col] >= 0).all() and (test_eng[col] >= 0).all():
        train_eng[col] = np.log1p(train_eng[col])
        test_eng[col]  = np.log1p(test_eng[col])

skew_after = train_eng[skewed.index].apply(lambda x: x.skew())
skew_compare = pd.DataFrame({"before": skew_before[skewed.index], "after": skew_after}).head(10)
print("\nSkewness before / after (top 10):")
skew_compare

### 📌 Encoding rationale
- **Ordinal** for quality columns because `Po < Fa < TA < Gd < Ex` is a meaningful order.
- **One-hot** for unordered categoricals (Neighborhood, RoofStyle, etc.) — no implicit ranking.
- **Log1p** on skewed numerics reduces the influence of extreme values and improves linearity.

---
## 6. Model Training & Cross-Validation Comparison

Comparing multiple model families tells us whether the problem is best served by linear methods or tree-based ensembles.  
Cross-validation gives a robust estimate of generalisation error, while the hold-out split provides a single sanity check.  
Timing each model helps us balance accuracy vs compute cost.

In [ ]:
# ── Train / validation split ─────────────────────────────────────────
X = train_eng.copy()
y = y_train_log.copy()

X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=SEED
)

In [ ]:
# ── Define models ────────────────────────────────────────────────────
models = {
    "Linear Regression": LinearRegression(),
    "Ridge":             Ridge(alpha=1.0, random_state=SEED),
    "Lasso":             Lasso(alpha=0.001, random_state=SEED, max_iter=10_000),
    "Random Forest":     RandomForestRegressor(n_estimators=200, random_state=SEED, n_jobs=-1),
    "XGBoost":           XGBRegressor(n_estimators=300, learning_rate=0.05,
                                      random_state=SEED, n_jobs=-1, verbosity=0),
}

kf = KFold(n_splits=CV_FOLDS, shuffle=True, random_state=SEED)

# ── Cross-validate each model ────────────────────────────────────────
records = []
for name, model in models.items():
    t0 = time.time()
    cv_scores = cross_val_score(model, X, y, cv=kf,
                                scoring="neg_root_mean_squared_error")
    elapsed = time.time() - t0

    cv_rmse = -cv_scores

    # Fit once on split for Val metrics
    model.fit(X_tr, y_tr)
    val_pred = model.predict(X_val)
    val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    val_r2   = r2_score(y_val, val_pred)

    records.append({
        "Model":         name,
        "CV RMSE (mean)": cv_rmse.mean(),
        "CV RMSE (std)":  cv_rmse.std(),
        "Val RMSE":      val_rmse,
        "Val R²":        val_r2,
        "Train time (s)": round(elapsed, 2),
    })

results_df = pd.DataFrame(records).sort_values("CV RMSE (mean)")
results_df

In [ ]:
# ── Bar chart: CV RMSE comparison ────────────────────────────────────
fig, ax = plt.subplots(figsize=(10, 5))
ax.barh(results_df["Model"], results_df["CV RMSE (mean)"],
        xerr=results_df["CV RMSE (std)"], color="steelblue",
        edgecolor="k", capsize=4)
ax.set_xlabel("CV RMSE (log scale)")
ax.set_ylabel("Model")
ax.set_title("Model Comparison — 5-Fold Cross-Validation RMSE")
ax.invert_yaxis()
plt.tight_layout()
fig.savefig(FIGURES_DIR / "model_comparison.png", dpi=150)
plt.show()

best_name = results_df.iloc[0]["Model"]
print(f"\n🏆 Best model: {best_name} (CV RMSE = {results_df.iloc[0]['CV RMSE (mean)']:.4f})")

- **XGBoost** and **Random Forest** typically outperform linear models because housing prices depend on non-linear interactions (e.g., quality × size × location).
- **Ridge/Lasso** still perform respectably thanks to our extensive feature engineering and skewness corrections.

---
## 7. Preprocessing Pipeline (sklearn Pipeline)

Wrapping preprocessing + model into a single `Pipeline` object prevents **data leakage**: the scaler/imputer is fitted only on the training fold during cross-validation, never on the validation fold.  
Pipelines also make deployment trivial — a single `pipeline.predict(new_data)` call handles everything.

In [ ]:
# ── Identify feature types in the engineered data ────────────────────
# After get_dummies, everything is numeric, so we use the raw data pipeline
# on the original df (before our manual engineering) to show the concept.

# Rebuild from raw data for Pipeline demo
raw_train = pd.read_csv(DATA_DIR / "train.csv")
raw_train = raw_train[raw_train["GrLivArea"] <= 4500].reset_index(drop=True)
raw_train["MSSubClass"] = raw_train["MSSubClass"].astype(str)
y_pipe = np.log1p(raw_train[TARGET_COL])
X_pipe = raw_train.drop(columns=[TARGET_COL, "Id"])

PIPE_NUM_FEATS = X_pipe.select_dtypes(include=np.number).columns.tolist()
PIPE_CAT_FEATS = X_pipe.select_dtypes(include="object").columns.tolist()

# ── Build ColumnTransformer ──────────────────────────────────────────
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  StandardScaler()),
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, PIPE_NUM_FEATS),
    ("cat", categorical_transformer, PIPE_CAT_FEATS),
])

# ── Create pipelines ────────────────────────────────────────────────
pipeline_ridge = Pipeline([("prep", preprocessor), ("model", Ridge(random_state=SEED))])
pipeline_xgb   = Pipeline([("prep", preprocessor), ("model", XGBRegressor(
    n_estimators=300, learning_rate=0.05, random_state=SEED, n_jobs=-1, verbosity=0))])

# ── Cross-validate pipelines ─────────────────────────────────────────
for name, pipe in [("Pipeline Ridge", pipeline_ridge), ("Pipeline XGBoost", pipeline_xgb)]:
    cv_scores = cross_val_score(pipe, X_pipe, y_pipe, cv=kf,
                                scoring="neg_root_mean_squared_error")
    rmse = -cv_scores
    print(f"{name:20s}  →  CV RMSE: {rmse.mean():.4f} ± {rmse.std():.4f}")

- Pipeline scores are slightly different from Section 6 because the manual feature engineering (TotalSF, etc.) is not included here — this is intentional to show the Pipeline concept.
- **Key benefit:** the `SimpleImputer` and `StandardScaler` are fitted only on each training fold, not the full dataset.

---
## 8. Hyperparameter Tuning (Best Model)

GridSearchCV exhaustively tests every combination in the search space with cross-validation.  
Even small RMSE improvements matter on Kaggle, where the leaderboard is dense.  
We tune the model that won in Section 6 — typically XGBoost.

In [ ]:
# ── GridSearchCV on our best-performing engineered features ────────
# Use the manually-engineered X/y from Section 6

param_grid_xgb = {
    "n_estimators":     [200, 300, 500],
    "max_depth":        [3, 4, 6],
    "learning_rate":    [0.01, 0.05, 0.1],
    "subsample":        [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

param_grid_ridge = {
    "alpha": [0.1, 1, 10, 50, 100, 500, 1000],
}

# Choose grid based on winner
if "XGBoost" in best_name:
    base = XGBRegressor(random_state=SEED, n_jobs=-1, verbosity=0)
    grid = param_grid_xgb
elif "Ridge" in best_name:
    base = Ridge(random_state=SEED)
    grid = param_grid_ridge
else:
    base = XGBRegressor(random_state=SEED, n_jobs=-1, verbosity=0)
    grid = param_grid_xgb
    print(f"Defaulting to XGBoost tuning (winner was {best_name})")

gs = GridSearchCV(
    estimator=base,
    param_grid=grid,
    cv=kf,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=1,
    refit=True,
)
gs.fit(X, y)

print(f"\n✅ Best params: {gs.best_params_}")
print(f"✅ Best CV RMSE: {-gs.best_score_:.4f}")

best_model = gs.best_estimator_

In [ ]:
# ── Tuned vs untuned comparison ──────────────────────────────────────
untuned_rmse = results_df[results_df["Model"] == best_name]["CV RMSE (mean)"].values[0]
tuned_rmse   = -gs.best_score_
improvement  = (untuned_rmse - tuned_rmse) / untuned_rmse * 100

print(f"Untuned CV RMSE: {untuned_rmse:.4f}")
print(f"Tuned   CV RMSE: {tuned_rmse:.4f}")
print(f"Improvement:     {improvement:.1f} %")

---
## 9. Model Interpretation & Feature Importance

Knowing *which* features drive predictions builds trust and domain understanding.  
Residual analysis checks whether our model errors are random (good) or patterned (bad — missed signal).  
Inspecting the worst predictions reveals edge cases the model struggles with.

In [ ]:
# ── A. Feature Importance (top 20) ───────────────────────────────────
best_model.fit(X_tr, y_tr)

if hasattr(best_model, "feature_importances_"):
    importances = best_model.feature_importances_
elif hasattr(best_model, "coef_"):
    importances = np.abs(best_model.coef_)
else:
    importances = np.zeros(X.shape[1])

feat_imp = pd.Series(importances, index=X.columns).nlargest(20).sort_values()

fig, ax = plt.subplots(figsize=(10, 8))
feat_imp.plot.barh(ax=ax, color="steelblue", edgecolor="k")
ax.set_title("Top 20 Feature Importances")
ax.set_xlabel("Importance")
ax.set_ylabel("Feature")
plt.tight_layout()
fig.savefig(FIGURES_DIR / "feature_importance.png", dpi=150)
plt.show()

In [ ]:
# ── B. Residuals Analysis ────────────────────────────────────────────
val_pred = best_model.predict(X_val)
residuals = y_val.values - val_pred

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Predicted vs Actual
axes[0].scatter(y_val, val_pred, alpha=0.4, c="steelblue", edgecolors="k", linewidths=0.3, s=20)
lims = [y_val.min() - 0.1, y_val.max() + 0.1]
axes[0].plot(lims, lims, "r--", lw=2, label="Ideal (y = x)")
axes[0].set_title("Predicted vs Actual (log scale)")
axes[0].set_xlabel("Actual log(SalePrice)")
axes[0].set_ylabel("Predicted log(SalePrice)")
axes[0].legend()

# Residuals distribution
sns.histplot(residuals, kde=True, ax=axes[1], color="steelblue")
axes[1].set_title("Residuals Distribution")
axes[1].set_xlabel("Residual")
axes[1].set_ylabel("Count")

# Residuals vs Predicted
axes[2].scatter(val_pred, residuals, alpha=0.4, c="steelblue", edgecolors="k", linewidths=0.3, s=20)
axes[2].axhline(0, color="red", ls="--", lw=1.5)
axes[2].set_title("Residuals vs Predicted")
axes[2].set_xlabel("Predicted log(SalePrice)")
axes[2].set_ylabel("Residual")

plt.tight_layout()
fig.savefig(FIGURES_DIR / "residuals_analysis.png", dpi=150)
plt.show()

In [ ]:
# ── C. Top 5 most under- and over-predicted houses ──────────────────
error_df = X_val.copy()
error_df["Actual_log"]    = y_val.values
error_df["Predicted_log"] = val_pred
error_df["Residual"]      = residuals
error_df["Actual_$"]      = np.expm1(y_val.values)
error_df["Predicted_$"]   = np.expm1(val_pred)

SHOW_COLS = ["OverallQual", "GrLivArea", "Actual_$", "Predicted_$", "Residual"]
show_cols_available = [c for c in SHOW_COLS if c in error_df.columns]

print("🔻 Top 5 OVER-predicted (model too high):")
print(error_df.nlargest(5, "Residual")[show_cols_available].to_string())

print("\n🔺 Top 5 UNDER-predicted (model too low):")
print(error_df.nsmallest(5, "Residual")[show_cols_available].to_string())

In [ ]:
# ── D. Ridge / Lasso Coefficients ────────────────────────────────────
ridge_model = Ridge(alpha=10, random_state=SEED)
ridge_model.fit(X_tr, y_tr)
coefs = pd.Series(ridge_model.coef_, index=X.columns)

top_pos = coefs.nlargest(15)
top_neg = coefs.nsmallest(15)
top_coefs = pd.concat([top_neg, top_pos]).sort_values()

fig, ax = plt.subplots(figsize=(10, 8))
colors = ["#e74c3c" if v < 0 else "#2ecc71" for v in top_coefs.values]
top_coefs.plot.barh(ax=ax, color=colors, edgecolor="k")
ax.set_title("Ridge Regression — Top 15 Positive & Negative Coefficients")
ax.set_xlabel("Coefficient")
ax.set_ylabel("Feature")
ax.axvline(0, color="k", lw=0.8)
plt.tight_layout()
fig.savefig(FIGURES_DIR / "ridge_coefficients.png", dpi=150)
plt.show()

### 📌 Coefficient interpretation
- **OverallQual** and **GrLivArea** have the largest positive coefficients — higher quality and more living space → higher price.
- **HouseAge** has a negative coefficient — older houses sell for less, all else equal.
- **Neighbourhood dummies** (e.g., NridgHt, StoneBr) capture location premiums that other features can't.

---
## 10. Final Predictions & Submission File

For the Kaggle submission, we refit the tuned model on the **entire** training set (no hold-out) to maximise available information.  
Since we trained on `log1p(SalePrice)`, we must apply `expm1` to convert predictions back to dollars.

In [ ]:
# ── Refit on full training data ──────────────────────────────────────
best_model.fit(X, y)

# ── Predict on test set ──────────────────────────────────────────────
test_pred_log = best_model.predict(test_eng)
test_pred_dollars = np.expm1(test_pred_log)

# ── Build submission ─────────────────────────────────────────────────
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": test_pred_dollars,
})

submission.to_csv(OUTPUT_DIR / "submission.csv", index=False)

# ── Sanity checks ────────────────────────────────────────────────────
assert (submission["SalePrice"] > 0).all(), "Negative prices detected!"
assert submission.shape[0] == len(test_ids), "Row count mismatch!"

print(f"Submission shape: {submission.shape}")
print(f"Predicted price range: ${submission['SalePrice'].min():,.0f} – ${submission['SalePrice'].max():,.0f}")
print()
submission.describe()

In [ ]:
submission.head(10)

> 📄 **`outputs/submission.csv`** has been saved and is ready for [Kaggle upload](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/submit).  
> Remember: the model predicts `log1p(SalePrice)` internally — `expm1` converts back to real dollars.

---
## 11. Key Takeaways & Next Steps

### Top 5 EDA findings
1. **OverallQual** is the single strongest predictor (correlation ≈ 0.79 with SalePrice).
2. **GrLivArea** shows a clear linear trend — but two outliers (>4500 sq ft, low price) needed removal.
3. **Neighbourhood** drives massive price variation: NoRidge/NridgHt/StoneBr vs MeadowV/IDOTRR.
4. Houses built after 2000 command significantly higher prices (modern construction premium).
5. "Missing" data in quality columns (PoolQC, Fence, etc.) actually means *absence of feature*, not unknown data.

### Best model
- **XGBoost** (tuned via GridSearchCV) achieved the best 5-fold CV RMSE on log(SalePrice).
- Residuals are approximately normally distributed with no strong patterns → good model fit.

### Top 3 most predictive features
1. **OverallQual** — Overall material and finish quality (ordinal 1–10).
2. **TotalSF** (engineered) — Combined total square footage (basement + 1st + 2nd floor).
3. **GrLivArea** — Above-ground living area in sq ft.

### Next steps
1. **Stacking / Blending** — Combine XGBoost + Ridge + LightGBM predictions with a meta-learner.
2. **SHAP values** — Instance-level feature explanations for model interpretability.
3. **Target Encoding** for high-cardinality categoricals (Neighborhood has 25 levels) — more efficient than one-hot.
4. **Title extraction** from features — explore interaction terms (OverallQual × GrLivArea).
5. **LightGBM / CatBoost** — Alternative gradient boosting implementations that often outperform XGBoost on tabular data.